<a href="https://colab.research.google.com/github/ufrpe-ensino/mt-aulas/blob/master/03a_Classifica%C3%A7%C3%A3oTextos_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de texto: supervisionado

A classificação de texto é uma tarefa de aprendizado de máquina que consiste em categorizar textos em classes predefinidas. Na abordagem **supervisionada**, o modelo é treinado com um conjunto de dados rotulados, onde cada texto já possui uma classe associada. Essa abordagem requer uma quantidade significativa de dados rotulados para alcançar bons resultados.

# Bibliotecas: Spacy e XGBoost

Neste notebook, utilizaremos a biblioteca SPACY (https://spacy.io/)

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
!pip install xgboost

# Carregando os dados

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/ufrpe-ensino/mt-aulas/refs/heads/master/data/spam_classification/spam.csv')
df

In [ ]:
df.isnull().values.any()

# Abordagem Supervisionada

## Pré-processamento

Remoção de pontuação

In [ ]:
import string

df['Text_no_punctuation_number'] = df['Text'].apply(lambda x: [token for token in x if token not in string.punctuation and not token.isnumeric()])
df['Text_no_punctuation_number'] = df['Text_no_punctuation_number'].apply(lambda x: ''.join(x))


Remoção de stopwords

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

df['Text_no_stopword'] = df['Text_no_punctuation_number'].apply(lambda x: [token.text.lower() for token in nlp(x) if (token.is_stop == False and len(token.text)>3)])
df['Text_no_stopword'] = df['Text_no_stopword'].apply(lambda x: ' '.join(x))

Lematização e remoção de stopwords

In [ ]:
df['Text_lemma_no_stopword'] = df['Text_no_stopword'].apply(lambda x: [token.lemma_ for token in nlp(x)])
df['Text_lemma_no_stopword'] = df['Text_lemma_no_stopword'].apply(lambda x: ' '.join(x))

Lematização

In [ ]:
df['Text_lemma'] = df['Text_no_punctuation_number'].apply(lambda x: [token.lemma_ for token in nlp(x)])
df['Text_lemma'] = df['Text_lemma'].apply(lambda x: ' '.join(x))

In [ ]:
df

Extração de features usando o TfidfVectorizer - neste exemplo iremos avaliar os textos com lemma e sem stopword

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Código para pegar os valores de uma coluna do dataframe (dataframe,nomedacoluna,.values)
X = df.Text_lemma_no_stopword.values

#Extração das features
vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(X)

X_tfidf = tfidf_model.transform(X)

print(X_tfidf[0,:])

## Função para treinamento e avaliação de vários modelos e métricas ao mesmo tempo usando treinamento e teste. Explicações dentro da função.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, make_scorer
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def run_exps_train_test(x_train: pd.DataFrame ,
             y_train: pd.DataFrame,
             x_test:  pd.DataFrame,
             y_test:  pd.DataFrame) -> pd.DataFrame:
    """
    Lightweight script to test many models and find winners
    :param x_train: train split
    :param y_train: training target vector
    :param x_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    """

    dfs = []
    
    #Modelos que serão avaliados (podem incluir quantos modelos quiserem)
    models = [
          ('LogReg', LogisticRegression()),
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC(kernel="linear")),
          ('MNB', MultinomialNB()),
          ('Adaboost', AdaBoostClassifier()),
          ('XGB', XGBClassifier())
        ]

    results = []
    names = []
    #Métricas que serão avaliadas (podem incluir quantos métricas quiserem)
    kappa_scorer = make_scorer(cohen_kappa_score)
    scoring = {
                'accuracy': 'accuracy',
                'precision_weighted': 'precision_weighted',
                'recall_weighted': 'recall_weighted',
                'f1_weighted': 'f1_weighted',
                'kappa' : kappa_scorer
                }

   #Nomes das classes, esse atributo é opcional, caso não seja incluido o modelo
   #vai apresentar os valores de 0-n onde n é o número de classes.
   # target_names = ['ham', 'spam']

    for name, model in models:
        #em alguns casos é interessante se criar um classificador para cada classe
        #caso seja o caso descomentar linha abaixo
        #model = OneVsRestClassifier(model)
    
        if name == 'XGB':
            # XGBoost nao aceita classes como string
            y_train = [1 if x == 'spam' else 0 for x in y_train]
            y_test  = [1 if x == 'spam' else 0 for x in y_test]
            
        clf = model.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(name)
        print(classification_report(y_test, y_pred))
       

## Recuperando classes das instâncias de treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split

Y = df.Class.values
tfidf_train, tfidf_test, class_train, class_test = train_test_split(
    X_tfidf, Y, test_size=0.25, stratify=Y
)

## Rodar função para treinamento e avaliação descrita acima.

In [ ]:
run_exps_train_test(tfidf_train, class_train, tfidf_test, class_test)

In [ ]:
from sklearn import model_selection

def run_exps_crossvalidation(x: pd.DataFrame ,
             y: pd.DataFrame) -> pd.DataFrame:
    """
    Lightweight script to test many models and find winners
    :param x: values vector
    :param y: target vector
    :return: DataFrame of predictions
    """

    dfs = []
    print("CARREGANDO MODELO")
    models = [
          ('LogReg', LogisticRegression()),
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('MNB', MultinomialNB()),
          ('Adaboost', AdaBoostClassifier()),
          ('XGB', XGBClassifier())
        ]

    results = []
    names = []
    kappa_scorer = make_scorer(cohen_kappa_score)
    scoring = {
                'accuracy': 'accuracy',
                'precision_weighted': 'precision_weighted',
                'recall_weighted': 'recall_weighted',
                'f1_weighted': 'f1_weighted',
                'kappa' : kappa_scorer
                }
    print("RODANDO")
    for name, model in models:
        print(name)

        kfold = model_selection.KFold(n_splits=10, shuffle=True)
        if name == 'XGB':
            # XGBoost nao aceita classes como string
            y = [1 if x == 'spam' else 0 for x in y]
        cv_results = model_selection.cross_validate(model, x, y, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)

    final = pd.concat(dfs, ignore_index=True)
    return final


In [ ]:
final = run_exps_crossvalidation(X_tfidf, Y)
final

In [ ]:
grouped = final[['test_accuracy','test_f1_weighted', 'test_kappa']].groupby(final['model'])
grouped.mean()

In [ ]:
grouped.std()

## Medindo a importância das features no classificador

Treinar o modelo de RandomForest

In [ ]:
model = RandomForestClassifier()
model.fit(X_tfidf, Y)

Extraindo a importância das características

In [ ]:
mdg_features = model.feature_importances_
mdg_features

Nomes e índices das features

In [ ]:
features_names = tfidf_model.get_feature_names_out()

feature_importance = pd.DataFrame(mdg_features,
                                   index = features_names,
                                   columns=['importance']).sort_values('importance',ascending=False)

index_feature_importance = pd.DataFrame(mdg_features,
                                   index = range(len(features_names)),
                                   columns=['importance']).sort_values('importance',ascending=False)

Montar o array de importância

In [ ]:
labels_features = feature_importance['importance'].index[:30]
indices_features = index_feature_importance['importance'].index[:30]
mdg_features = feature_importance['importance'].values[:30]

data = {"Variable": labels_features, "MDG": mdg_features}

df_feature_importance = pd.DataFrame(data)
df_feature_importance